The notebook gives the experiment reported in Fig. 7, where the APIS recovers images of the Set12 dataset that are densely corrupted with block noise.

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import cv2
from skimage import data
from numpy.fft import fft2, ifft2
from imglib import HT, obfuscation, noiseSP, rescale
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity as ssim
import time
import os
import pywt
from pywt import wavedec2, waverec2
from imglib import HT, HTHWS

In [2]:
def apis_fourier_wave_struc_sparse(noisy_img, ref_img, frac_k, frac_s, wname, max_iter=10):
    shape=noisy_img.shape
    noise = np.zeros(shape)
    loss=np.zeros(max_iter)
    
    l=int(np.log2(shape[0]))-int(np.log2(shape[0]*np.sqrt(frac_k)))
    frac=(np.power(2,2*l)*frac_k -1)/3

    for i in range(max_iter):
        clean_img= abs(ifft2(HT(fft2(noisy_img-noise),frac_s)))
        noise=waverec2(HTHWS(wavedec2(noisy_img-clean_img,wname,level=l),frac),wname)
        loss[i]=np.linalg.norm(clean_img-ref_img, ord='fro')
    return rescale(clean_img), loss

In [3]:
noise_name='block_noise/'

if not os.path.exists('output/'+str(noise_name)+'apis/'):
        os.makedirs('output/'+str(noise_name)+'apis/')

In [4]:
metrics=np.zeros(shape=(12,11))

for i in range(12):
    img=cv2.imread('input/Set12/'+str(i+1)+'.png',0)
    noisy_img=cv2.imread('input/'+str(noise_name)+str(i+1)+'.png',0)
    
    noisy_psnr=peak_signal_noise_ratio(img, noisy_img)
    noisy_ssim = ssim(img, noisy_img )
    
    metrics[i,0], metrics[i,1] = noisy_psnr, noisy_ssim
    print(f'img_no: {i+1}, noisy psnr={noisy_psnr:.2f}, ssim={noisy_ssim:.2f}')
    
    #---------------APIS------------------#
    frac_k=0.0001
    frac_s=0.06
    wname='haar'
    
    start_time=time.time()
    apis_img, loss=apis_fourier_wave_struc_sparse(noisy_img, img, frac_k, frac_s, wname, max_iter=2000)
    apis_time = time.time()-start_time
    apis_psnr=peak_signal_noise_ratio(img, apis_img )
    apis_ssim = ssim(img, apis_img )
    
    metrics[i,2], metrics[i,3], metrics[i,4] = apis_psnr, apis_ssim, apis_time
    print(f'apis psnr={apis_psnr:.2f}, ssim={apis_ssim:.2f}, time={apis_time:.2f}s')
    plt.imsave('output/'+str(noise_name)+'apis/'+str(i+1)+'.png',apis_img, cmap='gray')


img_no: 1, noisy psnr=11.03, ssim=0.68
apis psnr=20.04, ssim=0.79, time=82.09s
img_no: 2, noisy psnr=10.97, ssim=0.74
apis psnr=14.36, ssim=0.81, time=80.93s
img_no: 3, noisy psnr=11.15, ssim=0.69
apis psnr=22.73, ssim=0.79, time=79.34s
img_no: 4, noisy psnr=11.37, ssim=0.64
apis psnr=24.65, ssim=0.90, time=78.62s
img_no: 5, noisy psnr=11.09, ssim=0.67
apis psnr=25.51, ssim=0.76, time=79.53s
img_no: 6, noisy psnr=10.04, ssim=0.75
apis psnr=26.52, ssim=0.80, time=80.76s
img_no: 7, noisy psnr=11.20, ssim=0.69
apis psnr=26.25, ssim=0.86, time=78.57s
img_no: 8, noisy psnr=10.71, ssim=0.67
apis psnr=22.56, ssim=0.80, time=79.50s
img_no: 9, noisy psnr=10.92, ssim=0.65
apis psnr=21.43, ssim=0.70, time=80.32s
img_no: 10, noisy psnr=11.41, ssim=0.69
apis psnr=24.65, ssim=0.73, time=78.95s
img_no: 11, noisy psnr=10.70, ssim=0.59
apis psnr=24.03, ssim=0.75, time=83.32s
img_no: 12, noisy psnr=11.34, ssim=0.67
apis psnr=21.97, ssim=0.75, time=79.66s
